In [1]:
# Import the path of each sub-package.
from neuralplex_study.materials import MATERIALS_PATH
from neuralplex_study.methods import METHODS_PATH
from neuralplex_study.results import RESULTS_PATH

In [2]:
from random import random, randint
from typing import List, Dict, Self
import math
import json
from sklearn.metrics import r2_score

class Neuron:

    def __init__(self, m: float, b: float = 0, step: float = None, name=None):
        self.m = m  # weight
        self.b = b  # lift
        self.name = name
        self.step = step
        self.value = 0
        self.activation_count = 0
        self.neuronsRHS: List[Self] = []
        self.neuronsLHS: List[Self] = []
        self.activation: Dict[Self, float] = {}
        self.propagation: Dict[Self, float] = {}

    def connectRHS(self, neuronRHS: Self) -> None:
        if neuronRHS not in self.neuronsRHS:
            self.neuronsRHS.append(neuronRHS)
        if self not in neuronRHS.neuronsLHS:
            neuronRHS.connectLHS(self)

    def connectLHS(self, neuronLHS: Self) -> None:
        if neuronLHS not in self.neuronsLHS:
            self.neuronsLHS.append(neuronLHS)
        if self not in neuronLHS.neuronsRHS:
            neuronLHS.connectRHS(self)

    def disconnectRHS(self, neuronRHS: Self) -> None:
        if neuronRHS in self.neuronsRHS:
            self.neuronsRHS.remove(neuronRHS)
        if self in neuronRHS.neuronsLHS:
            neuronRHS.disconnectLHS(self)
        if len(self.neuronsRHS) == 0:
            for neuron in self.neuronsLHS:
                neuron.disconnectRHS(self)

    def disconnectLHS(self, neuronLHS: Self) -> None:
        if neuronLHS in self.neuronsLHS:
            self.neuronsLHS.remove(neuronLHS)
        if self in neuronLHS.neuronsRHS:
            neuronLHS.disconnectRHS(self)
        if len(self.neuronsLHS) == 0:
            for neuron in self.neuronsRHS:
                neuron.disconnectLHS(self)

    def activate(self, value: float, neuron: Self = None) -> None:
        if self.activation_count == 0:
            self.activation = {}
        self.activation_count = self.activation_count + 1
        self.activation.update({neuron: value})
        if len(self.neuronsLHS) == 0 or self.activation_count == len(self.neuronsLHS):
            self.value = self.m * sum(self.activation.values())
            for neuron in self.neuronsRHS:
                neuron.activate(self.value, self)
                # Under what conditions should a numeric activation value be propagated?
            self.activation_count = 0

    def propagate(self, error: float, neuron: Self = None) -> None:
        self.propagation.update({neuron: error})
        if len(self.neuronsRHS) == 0 or len(self.propagation) == len(self.neuronsRHS):
            for error in self.propagation.values():
                self.m = self.m - (error * self.step)
                # This is a primitive implementation; however, I have some ideas for how to improve it.
                # Please let me know if you have any thoughts on how this should be implemented.
            error_total = sum(self.propagation.values())
            for neuron in self.neuronsLHS:
                neuron_activation_value = self.activation[neuron]
                if (
                    neuron_activation_value > 0
                    and error_total > 0
                    or neuron_activation_value < 0
                    and error_total < 0
                ):
                    neuron.propagate(error_total, self)
                else:
                    neuron.propagate(math.copysign(1, error_total), self)
                # Likewise, the "backpropagation" still needs a lot of work.
            self.propagation = {}


class Layer:

    def __init__(self, neurons: List[Neuron], step: float = None):
        self.neurons = neurons
        self.step = step
        if not self.step is None:
            for neuron in self.neurons:
                if neuron.step is None:
                    neuron.step = self.step

    def connect(self, layer: Self) -> None:
        for p1 in self.neurons:
            for p2 in layer.neurons:
                p1.connectRHS(p2)


class Network:

    def __init__(self, layers: List[Layer]):
        self.layers = layers
        self.input_layer = layers[0]
        self.output_layer = layers[len(layers) - 1]
        for i in range(0, len(layers) - 1):
            l1 = layers[i]
            l2 = layers[i + 1]
            l1.connect(l2)

    def train(self, X_train: List[float], y_train: List[float]) -> None:
        if len(self.input_layer.neurons) != len(X_train):
            raise Exception(
                f"The length of the input training values, {len(X_train)}, is not equal to the length of input neurons: {len(self.input_layer.neurons)}"
            )
        if len(self.output_layer.neurons) != len(y_train):
            raise Exception(
                f"The length of the output training values, {len(y_train)}, is not equal to the length of output neurons: {len(self.output_layer.neurons)}"
            )

        # Activation Stage
        for i in range(0, len(X_train)):
            self.input_layer.neurons[i].activate(X_train[i], None)

        # Backpropagation Stage
        for i in range(0, len(y_train)):
            y_train = y_train[i]
            neuron = self.output_layer.neurons[i]
            neuron.propagate(neuron.value - y_train, None)

    def predict(self, X: List[float]) -> List[float]:
        if len(self.input_layer.neurons) != len(X):
            raise Exception(
                f"The length of the input values, {len(X)}, is not equal to the length of input neurons: {len(self.input_layer.neurons)}"
            )

        for i in range(0, len(X)):
            self.input_layer.neurons[i].activate(X[i])

        return [neuron.value for neuron in self.output_layer.neurons]

In [3]:
# class Neuron:

#     def __init__(self, m: float, b: float = 0, step: float = None, name=None, activation_function=None):
#         self.m = m  # weight
#         self.b = b  # bias
#         self.name = name
#         self.step = step
#         self.value = 0
#         self.activation_count = 0
#         self.neuronsRHS: List[Self] = []
#         self.neuronsLHS: List[Self] = []
#         self.activation: Dict[Self, float] = {}
#         self.propagation: Dict[Self, float] = {}
#         self.activation_function = activation_function or self.sigmoid  # Default to sigmoid

#     def connectRHS(self, neuronRHS: Self) -> None:
#         if neuronRHS not in self.neuronsRHS:
#             self.neuronsRHS.append(neuronRHS)
#         if self not in neuronRHS.neuronsLHS:
#             neuronRHS.connectLHS(self)

#     def connectLHS(self, neuronLHS: Self) -> None:
#         if neuronLHS not in self.neuronsLHS:
#             self.neuronsLHS.append(neuronLHS)
#         if self not in neuronLHS.neuronsRHS:
#             neuronLHS.connectRHS(self)

#     def activate(self, value: float, neuron: Self = None) -> None:
#         # Activation based on current value and weight
#         self.activation_count += 1
#         self.activation[neuron] = value

#         if len(self.neuronsLHS) == 0 or self.activation_count == len(self.neuronsLHS):
#             # Calculate the weighted sum
#             self.value = self.m * sum(self.activation.values()) + self.b
#             self.value = self.activation_function(self.value)  # Apply the activation function
#             for neuron in self.neuronsRHS:
#                 neuron.activate(self.value, self)
#             self.activation_count = 0

#     def propagate(self, error: float, neuron: Self = None) -> None:
#         # Use the derivative of the activation function for backpropagation
#         derivative = self.activation_derivative(self.value)

#         # Adjust the weight based on the error and learning rate
#         self.m -= self.step * error * derivative

#         # Sum all error contributions from LHS neurons
#         error_total = error * derivative
#         for neuron in self.neuronsLHS:
#             # Propagate the error backwards to the previous layer
#             neuron.propagate(error_total, self)

#         self.propagation.clear()

#     def activation_derivative(self, value: float) -> float:
#         """ Derivative of the activation function (Sigmoid by default) """
#         if self.activation_function == self.sigmoid:
#             return value * (1 - value)
#         elif self.activation_function == self.relu:
#             return 1 if value > 0 else 0
#         # Add more functions like tanh if necessary
#         return 1  # Default to identity function

#     def sigmoid(self, x: float) -> float:
#         """ Sigmoid activation function """
#         return 1 / (1 + math.exp(-x))

#     def relu(self, x: float) -> float:
#         """ ReLU activation function """
#         return max(0, x)

In [4]:
ITERATION = int(1e4)
STEP = 1e-4
l1 = Layer(neurons=[Neuron(m=random()) for i in range(0, 4)], step=STEP)
l2 = Layer(neurons=[Neuron(m=random()) for i in range(0, 8)], step=STEP)
l3 = Layer(neurons=[Neuron(m=random())], step=STEP)
n1 = Network([l1, l2, l3])

print("Training the model.")
for i in range(0, ITERATION):
    if i % 1e3 == 0:
        print(f"Training iteration: {i}")
    rn = randint(1, 15)
    b = [int(n) for n in bin(rn)[2:]]
    while len(b) < 4:
        b = [0] + b
    n1.train(b, [rn])

ys = []
ys_predicted = []
for i in range(1, 16):
    b = [int(n) for n in bin(i)[2:]]
    while len(b) < 4:
        b = [0] + b
    pn = n1.predict(b)
    ys.append(i)
    ys_predicted.append(pn[0])
    print(f"{i} input: {json.dumps(b)}, truth: {i} prediction: {json.dumps(pn)}")

R2 = r2_score(ys, ys_predicted)

print(f"R2: {R2}")

Training the model.
Training iteration: 0
Training iteration: 1000
Training iteration: 2000
Training iteration: 3000
Training iteration: 4000
Training iteration: 5000
Training iteration: 6000
Training iteration: 7000
Training iteration: 8000
Training iteration: 9000
1 input: [0, 0, 0, 1], truth: 1 prediction: [2.365934613334732]
2 input: [0, 0, 1, 0], truth: 2 prediction: [2.3219308212158394]
3 input: [0, 0, 1, 1], truth: 3 prediction: [4.687865434550571]
4 input: [0, 1, 0, 0], truth: 4 prediction: [3.3323830576151225]
5 input: [0, 1, 0, 1], truth: 5 prediction: [5.698317670949854]
6 input: [0, 1, 1, 0], truth: 6 prediction: [5.654313878830963]
7 input: [0, 1, 1, 1], truth: 7 prediction: [8.020248492165695]
8 input: [1, 0, 0, 0], truth: 8 prediction: [7.209885092570336]
9 input: [1, 0, 0, 1], truth: 9 prediction: [9.575819705905069]
10 input: [1, 0, 1, 0], truth: 10 prediction: [9.531815913786176]
11 input: [1, 0, 1, 1], truth: 11 prediction: [11.897750527120909]
12 input: [1, 1, 0, 0]